In [1]:
import os
import urllib.request
import json
import pymysql.cursors
import pandas as pd
from pymongo import MongoClient
from itertools import chain
from riotwatcher import RiotWatcher
from tqdm import tqdm
from requests.exceptions import HTTPError
from converters.data2files import get_runes_reforged_json
from converters.data2frames import game_to_dataframe as g2df
from sqlalchemy import create_engine
from datetime import datetime as dt, timedelta
import pickle
from config.constants import LEAGUES_DATA_DICT, SQL_LEAGUES_CONN, MONGODB_CREDENTIALS, \
    API_KEY, SOLOQ, REGIONS, CUSTOM_PARTICIPANT_COLS, STANDARD_POSITIONS, SCRIMS_POSITIONS_COLS, \
    TOURNAMENT_GAME_ENDPOINT, SQL_LEAGUES_ENGINE, EXCEL_EXPORT_PATH_MERGED, EXPORTS_DIR, EXCEL_EXPORT_PATH, \
    RIFT_GAMES_QUEUES, SLO, TOURNAMENT_TL_ENDPOINT, SQL_EXPORTS_ENGINE, API_KEY

In [2]:
rw = RiotWatcher(API_KEY)

In [4]:
tl = rw.match.timeline_by_match(match_id=3634493462, region='EUW1')

In [12]:
events = list(chain.from_iterable([f['events'] for f in tl['frames']]))

In [223]:
placed_ward_events = [event for event in events if event['type'] == 'WARD_PLACED']
killed_ward_events = [event for event in events if event['type'] == 'WARD_PLACED']

In [224]:
df_placed = pd.DataFrame([(event['creatorId'], event['wardType'], event['timestamp']) for event in placed_ward_events]).groupby([0, 1], as_index=False).count()
df_killed = pd.DataFrame([(event['creatorId'], event['wardType'], event['timestamp']) for event in killed_ward_events]).groupby([0, 1], as_index=False).count()

In [225]:
df_placed.rename(columns={0: 'participant_id', 1: 'ward_type', 2: 'count'}, inplace=True)
df_killed.rename(columns={0: 'participant_id', 1: 'ward_type', 2: 'count'}, inplace=True)

In [226]:
df_placed.set_index('participant_id', inplace=True)
df_killed.set_index('participant_id', inplace=True)

In [227]:
yt_p, cw_p, un_p, sw_p = df_placed.loc[df['ward_type'] == 'YELLOW_TRINKET'], df_placed.loc[df['ward_type'] == 'CONTROL_WARD'], df_placed.loc[df['ward_type'] == 'UNDEFINED'], df_placed.loc[df['ward_type'] == 'YELLOW_TRINKET']
yt_k, cw_k, un_k, sw_k = df_killed.loc[df['ward_type'] == 'YELLOW_TRINKET'], df_killed.loc[df['ward_type'] == 'CONTROL_WARD'], df_killed.loc[df['ward_type'] == 'UNDEFINED'], df_killed.loc[df['ward_type'] == 'YELLOW_TRINKET']

In [228]:
df1 = pd.concat([yt_p, cw_p, un_p, sw_p], axis=1).drop('ward_type', axis=1)
df2 = pd.concat([yt_k, cw_k, un_k, sw_k], axis=1).drop('ward_type', axis=1)

In [229]:
ward_cols = ['yellow_trinkets', 'control_wards', 'undefined', 'sight_wards']
placed_cols = [col + '_placed' for col in ward_cols]
killed_cols = [col + '_killed' for col in ward_cols]

In [230]:
df1.columns = placed_cols
df2.columns = killed_cols

In [231]:
df1.reset_index(inplace=True)
df2.reset_index(inplace=True)

In [232]:
df1.participant_id = df1.participant_id - 1
df2.participant_id = df2.participant_id - 1

In [233]:
df1.set_index('participant_id', inplace=True)
df2.set_index('participant_id', inplace=True)

In [234]:
df_result = pd.concat([df1, df2], axis=1).fillna(0)

,yellow_trinkets_placed,control_wards_placed,undefined_placed,sight_wards_placed,yellow_trinkets_killed,control_wards_killed,undefined_killed,sight_wards_killed
participant_id,,,,,,,,
0,2,0,0,2,2,0,0,2
1,4,0,0,4,4,0,0,4
2,2,0,3,2,2,0,3,2
3,2,1,2,2,2,1,2,2
4,3,2,0,3,3,2,0,3
5,3,1,0,3,3,1,0,3
6,3,3,0,3,3,3,0,3
8,3,0,3,3,3,0,3,3
9,2,2,14,2,2,2,14,2
